In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.action_chains import ScrollOrigin
from selenium.webdriver.common.action_chains import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from time import sleep
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import random

import datetime 
# import utils

import os 
import logging
import json
import datetime 
import sys

def setting_log():
    '''
        해당 시스템 log 설정 
    '''
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y_%m_%d_%H_%M_%S")
    logging.basicConfig(filename=f"./save_log/{formatted_datetime}.log",
                    format='%(asctime)s %(levelname)s:%(message)s',
                    level=logging.INFO,
                    datefmt='%Y/%m/%d %I:%M:%S %p', )

# 하나의 동을 끝내면 저장하는 폴더
folder_path = "./record_info"
log_path = "./save_log"
data_path = "./datas"
csv_path ="./csv_folder"

# 크롤링 저장 폴더가 없으면 생성
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"{folder_path} 폴더가 생성되었습니다.")
else:
    print(f"{folder_path} 폴더가 이미 존재합니다.")

#
#  로그 폴더가 없으면 생성 
if not os.path.exists(log_path):
    os.makedirs(log_path)
    print(f"{log_path} 폴더가 생성되었습니다.")
else:
    print(f"{log_path} 폴더가 이미 존재합니다.")


# 로그 폴더가 없으면 생성 
if not os.path.exists(csv_path):
    os.makedirs(csv_path)
    print(f"{csv_path} 폴더가 생성되었습니다.")
else:
    print(f"{csv_path} 폴더가 이미 존재합니다.")

# log 설정 세팅
setting_log() 

./record_info 폴더가 이미 존재합니다.
./save_log 폴더가 이미 존재합니다.
./csv_folder 폴더가 이미 존재합니다.


In [ ]:
def init_():
    options = webdriver.ChromeOptions()
    options.add_argument("window-size=1280,800")  # 너비=1280, 높이=800

    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(1) # 암시적 대기 시간 설정 
    driver.set_page_load_timeout(20) # 페이지 로드 대기 시간 설정 -> 해당 시간이 지나면 TimeException 발생 
    
    driver.get("https://www.disco.re/") # url 실행
    try:
        close = driver.find_element(By.CLASS_NAME, "disco-welcome-button.disco-welcome-close")
        ActionChains(driver).click(close).perform()
        return driver
    except:
        return driver

def settings(driver , text = "제주도 서귀포시 남원읍"):
    '''
        text : 검색하고자 하는 주소, 시 구 동까지 적어주면 좋음 
    
    '''
    # 1. 돋보기 누르기 
    driver.find_element(By.ID, "dsv_search_btn").click()  # ID로 입력란 찾기
    time.sleep(1)

    # 2. 검색창 입력 
    search_input = driver.find_element(By.ID, "top_search_ds_input")  # ID로 입력란 찾기
    search_input.click()
    search_input.send_keys(f"{text}")  # input_value를 작성란에 입력
    time.sleep(0.5)

    # 3. 첫번째 row 클릭 
    driver.find_element(By.CLASS_NAME, "autocomplete-suggestion").click()  # ID로 입력란 찾기
    # 4. 매물 버튼 클릭 
    sale_button = driver.find_element(By.XPATH, '//*[@data-value="sale"]')
    sale_button.click()

    # 거래방식 -> 매물로 조정
    driver.find_element(By.ID, "dealDropdown").click() 
    driver.find_element(By.XPATH, '//*[@class="dropdown-content show"]//li[@data-value="0"]').click()
    
def find_other_location(driver):
    # 무한 스크롤을 위해 스크롤의 범위 설정
    scrollable_element = driver.find_element(By.ID, "dsv_filter_tabmenu_sub")

    while True:
        for _ in range(100):
            driver.execute_script("arguments[0].scrollIntoView();", scrollable_element) # 스크롤 

        # 지역이 바뀌는 경우까지 스크롤을 하였는가 체크 
        # 0이 되는 시점이 다른 지역을 찾는 경우임
        element = driver.find_elements(By.XPATH, '//*[@data-area-order="0"]')
        # 다른 지역이 나오기 시작 
        if element != []:
            break
    
    return driver.find_elements(By.XPATH, '//*[@data-area-order="1"]')


def onclick_url_extract(find_list):
    js_url = []
    uploads = []
    for i in find_list:
        text = i.find_element(By.CLASS_NAME,"feed-content.feed-clickable").get_attribute("onclick")
        upload_ = i.find_element(By.CLASS_NAME,"feed-head-content-info").text
        # print(text)

        # 텍스트 파일에 append
        with open("js_url.txt", "a", encoding="utf-8") as f:
            f.write(text + "\n")
            
        js_url.append(text)
        uploads.append(upload_)
    
    if len(js_url) == 0:
        with open("js_url.txt", "a", encoding="utf-8") as f:
            pass
        
    return js_url , uploads

def start_crawling(driver, js_url , save_txt = "./js_url.txt"):
    results = []

    with open(f"{save_txt}", "r", encoding="utf-8") as f:
        js_lines = f.readlines()
    index = 0
    end = len(js_lines) 
    while True :   
        if index >= end :
            break

        script = js_lines[index]
        try:
        
            driver.execute_script(script)
            time.sleep(0.5)
            # 주소, 금액 정보 얻기 
            address = driver.find_element(By.CLASS_NAME, "dsv-mc-address").text
            cost = driver.find_element(By.CLASS_NAME, "dsv-mc-price").text

            # 정보 글고 뒤로 이동 
            # driver.find_element(By.CLASS_NAME, "dsc-head-left-align").click()
            print(index , address)
            results.append([address,cost])
            index += 1  
        except:
            print(f"{index} 오류, 다시 실행 ")
            time.sleep(5)

    return results
    

def load_gu_list(path = "./data"):
    # js 목록 txt 쓰기 
    with open(f"{path}/gu_list.txt", "r", encoding="utf-8") as f:
        gu_list = f.readlines()
    return gu_list

def post_processing(results, upload_):
    updated_data = [row + [date] for row, date in zip(results, upload_)]
    results_df = pd.DataFrame(updated_data,columns = ["위치","가격","확인날짜"])
    results_df["가격"] = results_df["가격"].apply(lambda x : x.split("(")[0])

    return results_df
 

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
def post_processing2(x):
    # 현재 시간
    now = datetime.now()
    pivot = x.split(" ")[0]
    if "분" in pivot:
        input_ = int(pivot.split("분")[0])
        date_ = "확인매물"+(now - timedelta(minutes=input_)).strftime('%Y.%m.%d.')
    elif "시간" in pivot:
        input_ = int(pivot.split("시간")[0])
        date_ = "확인매물"+(now - timedelta(hours=input_)).strftime('%Y.%m.%d.')
    elif "일" in pivot:
        input_ = int(pivot.split("일")[0])
        date_ = "확인매물"+(now - timedelta(days=input_)).strftime('%Y.%m.%d.')
    elif "주" in pivot:
        input_ = int(pivot.split("주")[0])
        date_ = "확인매물"+(now - timedelta(weeks=input_)).strftime('%Y.%m.%d.')
    elif "개월" in pivot:
        input_ = int(pivot.split("개월")[0])
        date_ = "확인매물"+(now - relativedelta(months=input_)).strftime('%Y.%m.%d.')
    return date_




# js 목록 txt 쓰기 
with open("./datas/gu_list.txt", "r", encoding="utf-8") as f:
    gu_list = f.readlines()


import os 
def get_max_number_csv_sorted(folder_path):
    numbers = []
    if len(os.listdir(folder_path)) == 0 :
        return 0
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            try:
                number = int(os.path.splitext(filename)[0])  # '1.csv' → 1
                numbers.append(number)
            except ValueError:
                continue  # 숫자가 아닌 파일명은 건너뜀

    if not numbers:
        return None  # 숫자가 없을 경우

    numbers.sort()
    return numbers[-1] + 1  # 가장 큰 숫자
index = get_max_number_csv_sorted("./csv_folder") 


while True:
    enter_gu = gu_list[index]    
    print(enter_gu)
    # driver 정의
    driver = init_()
    # 크롤링 할 수 있도록 '주소' 세팅 
    settings(driver, enter_gu)
    time.sleep(0.5)
    # settings(driver, "서울특별시 양천구 신월동")
    
    # 1. 크롤링 해야 할 리스트 추출 --> 여기까지는 문제없이 진행됨
    find_list = find_other_location(driver)
    time.sleep(0.5)
    # break
    # 2. java script 링크 기록 -> 중간에 딜레이때문에 멈추는 경우가 존재하므로
    js_url , upload_ = onclick_url_extract(find_list)
    time.sleep(0.5)

    # multiprocessing을 이용하면 훨씬 빠를 듯 ( 현재는 single processing )
    results = start_crawling(driver , js_url  )

    # 기존 js url 제거 ( 다른 구 새로 생성 )
    os.remove("./js_url.txt")

    # make csv , 후처리
    csv_ = post_processing(results, upload_)

    csv_["확인날짜"] = csv_["확인날짜"].apply(lambda x : post_processing2(x))

    csv_.to_csv(f"./csv_folder/{index}.csv", index = False)

    driver.close()

서울특별시 마포구 서교동

0 서울특별시 마포구 서교동 452-5
1 서울특별시 마포구 서교동
2 서울특별시 마포구 서교동 366-14
3 서울특별시 마포구 서교동 368-22
4 서울특별시 마포구 서교동 360-2
5 서울특별시 마포구 서교동 327-23
6 서울특별시 마포구 서교동 482-8
7 서울특별시 마포구 서교동 358-121
8 서울특별시 마포구 서교동 327-23
9 서울특별시 마포구 서교동 328-7
10 서울특별시 마포구 서교동 326-22
11 서울특별시 마포구 서교동 442-29
12 서울특별시 마포구 서교동 336-2
13 서울특별시 마포구 서교동 334-4
14 서울특별시 마포구 서교동 328-24
15 서울특별시 마포구 서교동 401-14
16 서울특별시 마포구 서교동 401-14
17 서울특별시 마포구 서교동 375-45
18 서울특별시 마포구 서교동 354-12
19 서울특별시 마포구 서교동 247-119
20 서울특별시 마포구 서교동 448-37
21 서울특별시 마포구 서교동 403-11
22 서울특별시 마포구 서교동 247-119
23 서울특별시 마포구 서교동 247-119
24 서울특별시 마포구 서교동 247-119
25 서울특별시 마포구 서교동 247-119
26 서울특별시 마포구 서교동 247-119
27 서울특별시 마포구 서교동 336-21
28 서울특별시 마포구 서교동 247-119
29 서울특별시 마포구 서교동 336-21
30 서울특별시 마포구 서교동 358-91
31 서울특별시 마포구 서교동 360-6
32 서울특별시 마포구 서교동 368-9
33 서울특별시 마포구 서교동 370-17
34 서울특별시 마포구 서교동 370-12
35 서울특별시 마포구 서교동 449-6
36 서울특별시 마포구 서교동 463-2
37 서울특별시 마포구 서교동 458-1
38 서울특별시 마포구 서교동 459-5
39 서울특별시 마포구 서교동 480-9
40 서울특별시 마포구 서교동 480-17
41 서울특별시 마포구 서교동 480-27

UnboundLocalError: cannot access local variable 'date_' where it is not associated with a value